In [145]:
import logging
from pathlib import Path
from typing import Dict

# import warnings filter
from warnings import simplefilter

import mlflow
import numpy as np
import pandas as pd
from dotenv import dotenv_values
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# from config import logger
from lib.data_handling import CustomSpectralPipeline, load_split_data  # type: ignore
from lib.outlier_removal import (
    calculate_leverage_residuals,
    identify_outliers,
    plot_leverage_residuals,
)
from lib.reproduction import (
    major_oxides,
    masks,
    optimized_blending_ranges,
    oxide_ranges,
    paper_individual_sm_rmses,
    spectrometer_wavelength_ranges,
    training_info,
)
from lib.utils import custom_kfold_cross_validation, filter_data_by_compositional_range
from PLS_SM.inference import predict_composition_with_blending

# ignore all future warnings
simplefilter(action="ignore", category=FutureWarning)

env = dotenv_values()
comp_data_loc = env.get("COMPOSITION_DATA_PATH")
dataset_loc = env.get("DATA_PATH")

if not comp_data_loc:
    print("Please set COMPOSITION_DATA_PATH in .env file")
    exit(1)

if not dataset_loc:
    print("Please set DATA_PATH in .env file")
    exit(1)

logger = logging.getLogger("train")

mlflow.set_tracking_uri("http://localhost:5000")

preformatted_data_path = Path("./data/_preformatted_sm/")
train_path = preformatted_data_path / "train.csv"
test_path = preformatted_data_path / "test.csv"

train_n1_path = preformatted_data_path / "train_n1.csv"
train_n3_path = preformatted_data_path / "train_n3.csv"
test_n1_path = preformatted_data_path / "test_n1.csv"
test_n3_path = preformatted_data_path / "test_n3.csv"

if (
    not preformatted_data_path.exists()
    or not train_path.exists()
    or not test_path.exists()
):
    take_samples = None

    logger.info("Loading data from location: %s", dataset_loc)
    # data = load_data(str(dataset_loc))
    train_data, test_data = load_split_data(
        str(dataset_loc), split_loc="./train_test_split.csv", average_shots=True
    )
    logger.info("Data loaded successfully.")

    logger.info("Initializing CustomSpectralPipeline.")
    pipeline = CustomSpectralPipeline(
        masks=masks,
        composition_data_loc=comp_data_loc,
        major_oxides=major_oxides,
    )
    logger.info("Pipeline initialized. Fitting and transforming data.")
    train_processed = pipeline.fit_transform(train_data)
    test_processed = pipeline.fit_transform(test_data)
    logger.info("Data processing complete.")

    preformatted_data_path.mkdir(parents=True, exist_ok=True)

    train_processed.to_csv(train_path, index=False)
    test_processed.to_csv(test_path, index=False)
else:
    logger.info("Loading preformatted data from location: %s", preformatted_data_path)
    train_processed = pd.read_csv(train_path)
    test_processed = pd.read_csv(test_path)

In [146]:
from sklearn.base import BaseEstimator, TransformerMixin
import enum
from typing import Dict, Tuple

class Norm3Scaler(BaseEstimator, TransformerMixin):
    def __init__(
        self, wavelength_ranges: Dict[str, Tuple[float, float]], reshaped=False
    ):
        self.scaler = (
            Norm3ScalerReshapedData(wavelength_ranges)
            if reshaped
            else Norm3ScalerOriginalData(wavelength_ranges)
        )

    def fit(self, df):
        return self.scaler.fit(df)

    def transform(self, df):
        return self.scaler.transform(df)


class Norm3ScalerOriginalData(BaseEstimator, TransformerMixin):
    def __init__(self, wavelength_ranges: Dict[str, Tuple[float, float]]):
        self.wavelength_ranges = wavelength_ranges
        self.totals = None

    def fit(self, df):
        """
        Compute the total intensity for each spectrometer range.
        """
        self.totals = {}
        shot_columns = df.columns[df.columns.str.startswith("shot")]
        for key, (start, end) in self.wavelength_ranges.items():
            mask = (df["wave"] >= start) & (df["wave"] <= end)
            self.totals[key] = df.loc[mask, shot_columns].sum().sum()
        return self

    def transform(self, df):
        """
        Apply norm3 normalization to the DataFrame.
        """
        if self.totals is None:
            raise ValueError("The fit method must be called before transform.")

        shot_columns = df.columns[df.columns.str.startswith("shot")]
        for key, (start, end) in self.wavelength_ranges.items():
            mask = (df["wave"] >= start) & (df["wave"] <= end)
            df.loc[mask, shot_columns] = df.loc[mask, shot_columns].div(
                self.totals[key], axis=1
            )
        return df


class Norm3ScalerReshapedData(BaseEstimator, TransformerMixin):
    """
    This class is used to normalize the data in the same way as the
    Norm3Scaler class, but it is used for the reshaped data. This is
    necessary because the reshaped data has a different format than
    the original data.

    The reshaped data has the following format:
    - Each row represents a single shot
    - Each column represents a single wavelength
    - The column names are the wavelengths
    """

    def __init__(self, wavelength_ranges: Dict[str, Tuple[float, float]]):
        self.wavelength_ranges = wavelength_ranges
        self.totals = None

    def fit(self, df):
        """
        Compute the total intensity for each spectrometer range.
        """
        self.totals = {}

        # Convert column names to floats. If conversion fails, assign NaN
        float_cols = pd.to_numeric(df.columns, errors='coerce')

        # Initialize an empty set to keep track of all selected columns
        selected_columns_set = set()

        for key, (start, end) in self.wavelength_ranges.items():
            # Use boolean indexing to select columns in the specified range
            selected_columns = df.columns[(float_cols >= start) & (float_cols <= end)]
            selected_columns_set.update(selected_columns)

            # Compute the sum of intensities in these columns
            self.totals[key] = df[selected_columns].sum().sum()

        # Identify columns that are not in any range
        all_columns_set = set(df.columns)
        out_of_range_columns = all_columns_set - selected_columns_set

        # Handle or report out-of-range columns
        if out_of_range_columns:
            print("Warning: There are columns outside of the specified ranges:", out_of_range_columns)
        else:
            print("All columns fall within the specified ranges.")
        
        assert len(self.totals) == 3, "Expected 3 spectrometer ranges"
        print(self.totals)
        sum_of_totals = sum(self.totals.values())
        print(sum_of_totals)
        return self

    def transform(self, df):
        """
        Apply norm3 normalization to the DataFrame.
        """
        if self.totals is None:
            raise ValueError("The fit method must be called before transform.")

        for key, (start, end) in self.wavelength_ranges.items():
            # Select columns in the specified range and ignore non-float columns
            selected_columns = []
            for col in df.columns:
                try:
                    if start <= float(col) <= end:
                        selected_columns.append(col)
                except ValueError:
                    # Ignore columns that cannot be converted to float
                    continue

            # Normalize intensities in these columns
            df[selected_columns] = df[selected_columns].div(self.totals[key], axis=0)
        return df


drop_cols = major_oxides + ["Sample Name", "ID"]

In [147]:
train_processed.drop(columns=drop_cols).T.describe()

,0,1,2,3,4,5,6,7,8,9,...,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537
count,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,...,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03
mean,9.127752e+10,9.184484e+10,1.018471e+11,1.042859e+11,1.037726e+11,8.229076e+10,8.437058e+10,8.061901e+10,7.986165e+10,8.448846e+10,...,7.238601e+10,8.496803e+10,7.935180e+10,7.460456e+10,7.349004e+10,1.114774e+11,1.191165e+11,1.201812e+11,1.171465e+11,1.078451e+11
std,2.246761e+11,2.295047e+11,2.567401e+11,2.499579e+11,2.534807e+11,1.914302e+11,2.054886e+11,1.954229e+11,1.924457e+11,2.051457e+11,...,1.569500e+11,1.870417e+11,1.702073e+11,1.619009e+11,1.616472e+11,2.533937e+11,2.452844e+11,2.647352e+11,2.537774e+11,2.413013e+11
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.512402e+09,7.645420e+09,8.049826e+09,8.411631e+09,8.322361e+09,8.571419e+09,8.231139e+09,7.810937e+09,7.749104e+09,8.166487e+09,...,8.322698e+09,9.212789e+09,8.629386e+09,8.210420e+09,8.484050e+09,1.125110e+10,1.193616e+10,1.100446e+10,1.108986e+10,1.091029e+10
50%,2.840975e+10,2.928102e+10,3.139123e+10,3.318297e+10,3.260693e+10,2.988350e+10,2.866448e+10,2.761184e+10,2.661856e+10,2.895692e+10,...,2.756152e+10,3.093531e+10,2.947862e+10,2.746075e+10,2.812013e+10,3.814736e+10,4.072848e+10,3.741068e+10,3.799930e+10,3.587706e+10
75%,8.565343e+10,8.669389e+10,9.354647e+10,9.793859e+10,9.869873e+10,8.080299e+10,8.044700e+10,7.820202e+10,7.809604e+10,8.128686e+10,...,7.391903e+10,8.236654e+10,7.887892e+10,7.465425e+10,7.441198e+10,1.098490e+11,1.134459e+11,1.100047e+11,1.093615e+11,1.066248e+11
max,4.265688e+12,4.592867e+12,5.009349e+12,4.616491e+12,4.809581e+12,2.832522e+12,3.141137e+12,2.822526e+12,2.953743e+12,2.980790e+12,...,2.513769e+12,2.527600e+12,2.305864e+12,2.346277e+12,2.505947e+12,4.375193e+12,3.105788e+12,3.561074e+12,3.592492e+12,4.133483e+12


In [148]:
train_processed.drop(columns=drop_cols).T.sum().sum()

9.881857093203355e+17

In [149]:
train_cols = train_processed.columns

In [150]:
scaler = Norm3Scaler(wavelength_ranges=spectrometer_wavelength_ranges, reshaped=True)
train_processed = scaler.fit_transform(train_processed)

{'UV': 6.176395627352914e+17, 'VIO': 2.6972361404241165e+17, 'VNIR': 5.452579200577558e+16}
9.418889687834787e+17


In [151]:
train_processed = pd.DataFrame(train_processed, columns=train_cols)

In [152]:
totals = {}

# Convert column names to floats. If conversion fails, assign NaN
float_cols = pd.to_numeric(train_processed.columns, errors='coerce')

# Initialize an empty set to keep track of all selected columns
selected_columns_set = set()

for key, (start, end) in spectrometer_wavelength_ranges.items():
    # Use boolean indexing to select columns in the specified range
    selected_columns = train_processed.columns[(float_cols >= start) & (float_cols <= end)]
    selected_columns_set.update(selected_columns)

    # Compute the sum of intensities in these columns
    totals[key] = train_processed[selected_columns].sum().sum()

totals

{'UV': 1.0, 'VIO': 1.0, 'VNIR': 0.9999999999999999}

In [153]:
train_processed.head()

,246.688,246.741,246.79401,246.847,246.89999,246.953,247.007,247.06,247.11301,247.166,...,SiO2,TiO2,Al2O3,FeOT,MgO,CaO,Na2O,K2O,Sample Name,ID
0,2.940677e-07,2.628665e-07,2.218354e-07,1.800553e-07,1.617129e-07,1.721856e-07,1.985312e-07,2.240468e-07,2.363474e-07,2.364594e-07,...,79.35,0.3,9.95,2.18,1.0,1.2,2.75,1.84,201426,201426_2013_11_06_161336_ccs
1,3.245969e-07,2.903669e-07,2.464502e-07,2.033082e-07,1.835096e-07,1.933978e-07,2.212464e-07,2.489649e-07,2.617880e-07,2.600798e-07,...,79.35,0.3,9.95,2.18,1.0,1.2,2.75,1.84,201426,201426_2013_11_06_161134_ccs
2,3.407285e-07,3.059225e-07,2.603263e-07,2.124742e-07,1.869284e-07,1.941817e-07,2.236271e-07,2.541700e-07,2.677841e-07,2.649219e-07,...,79.35,0.3,9.95,2.18,1.0,1.2,2.75,1.84,201426,201426_2013_11_06_162544_ccs
3,4.088900e-07,3.678309e-07,3.126661e-07,2.532956e-07,2.223067e-07,2.316652e-07,2.680076e-07,3.052780e-07,3.215675e-07,3.166276e-07,...,79.35,0.3,9.95,2.18,1.0,1.2,2.75,1.84,201426,201426_2013_11_06_161514_ccs
4,3.518082e-07,3.182807e-07,2.736818e-07,2.256416e-07,2.033465e-07,2.133621e-07,2.437775e-07,2.755768e-07,2.890620e-07,2.842021e-07,...,79.35,0.3,9.95,2.18,1.0,1.2,2.75,1.84,201426,201426_2013_11_06_160941_ccs


In [154]:
train_processed = train_processed.drop(columns=drop_cols).T

In [155]:
train_processed.head()

,0,1,2,3,4,5,6,7,8,9,...,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537
246.688,2.940677e-07,3.245969e-07,3.407285e-07,4.088900e-07,3.518082e-07,1.070512e-07,9.124374e-08,9.871574e-08,1.325500e-07,9.904988e-08,...,1.279879e-07,2.234684e-07,2.179040e-07,1.749742e-07,1.124082e-07,3.830759e-07,8.642088e-07,1.056854e-06,7.674444e-07,4.657276e-07
246.741,2.628665e-07,2.903669e-07,3.059225e-07,3.678309e-07,3.182807e-07,1.007656e-07,8.187881e-08,8.981021e-08,1.191105e-07,9.113817e-08,...,1.163014e-07,1.956847e-07,1.921276e-07,1.542728e-07,1.025549e-07,3.428793e-07,7.925558e-07,9.650137e-07,7.127930e-07,4.209400e-07
246.79401,2.218354e-07,2.464502e-07,2.603263e-07,3.126661e-07,2.736818e-07,9.265705e-08,7.093990e-08,7.881457e-08,1.010559e-07,8.175204e-08,...,1.016951e-07,1.594341e-07,1.598625e-07,1.282239e-07,8.879420e-08,2.827055e-07,6.953482e-07,8.401200e-07,6.317739e-07,3.567920e-07
246.847,1.800553e-07,2.033082e-07,2.124742e-07,2.532956e-07,2.256416e-07,8.533125e-08,6.206551e-08,7.096315e-08,8.345638e-08,7.225694e-08,...,8.695179e-08,1.242011e-07,1.280734e-07,1.048691e-07,7.530043e-08,2.178431e-07,5.759945e-07,6.836881e-07,5.303834e-07,2.824148e-07
246.89999,1.617129e-07,1.835096e-07,1.869284e-07,2.223067e-07,2.033465e-07,8.440418e-08,6.131329e-08,6.973670e-08,7.706504e-08,7.170758e-08,...,8.154775e-08,1.106936e-07,1.164410e-07,9.707695e-08,6.990602e-08,1.933899e-07,5.245905e-07,6.236243e-07,4.841648e-07,2.516249e-07


In [156]:
train_processed.describe()

,0,1,2,3,4,5,6,7,8,9,...,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537
count,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,...,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03,5.485000e+03
mean,3.972716e+09,3.914676e+09,4.317048e+09,4.458777e+09,4.566679e+09,4.136896e+09,3.847693e+09,3.549741e+09,3.472842e+09,3.742581e+09,...,3.420914e+09,3.747758e+09,3.586045e+09,3.415412e+09,3.483498e+09,6.134766e+09,6.680944e+09,5.490599e+09,6.750890e+09,5.648024e+09
std,2.185285e+10,2.142886e+10,2.485019e+10,2.555627e+10,2.526124e+10,2.158375e+10,2.072295e+10,1.865996e+10,1.816874e+10,1.980333e+10,...,1.797825e+10,1.967217e+10,1.877467e+10,1.792134e+10,1.846440e+10,3.405116e+10,3.803378e+10,2.856613e+10,3.858137e+10,3.004021e+10
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.892421e-08,4.888780e-08,5.379712e-08,5.568947e-08,5.517769e-08,5.201454e-08,4.879736e-08,4.663180e-08,4.494762e-08,4.913957e-08,...,4.774892e-08,5.464539e-08,5.155393e-08,4.827826e-08,4.941489e-08,6.589037e-08,7.250331e-08,6.630114e-08,6.606661e-08,6.402867e-08
50%,1.321549e-07,1.344380e-07,1.435041e-07,1.531491e-07,1.493125e-07,1.409196e-07,1.322831e-07,1.265039e-07,1.266605e-07,1.311660e-07,...,1.242498e-07,1.389097e-07,1.316795e-07,1.251013e-07,1.278612e-07,1.848263e-07,1.904109e-07,1.829485e-07,1.820597e-07,1.750430e-07
75%,3.604905e-07,3.597304e-07,3.960229e-07,4.062325e-07,4.078397e-07,3.456523e-07,3.551010e-07,3.358881e-07,3.335302e-07,3.517755e-07,...,3.180134e-07,3.567809e-07,3.383934e-07,3.146635e-07,3.294619e-07,4.755008e-07,5.017162e-07,4.712282e-07,4.972831e-07,4.562919e-07
max,3.729072e+11,3.633477e+11,4.572484e+11,4.713516e+11,4.371172e+11,2.486660e+11,3.197406e+11,2.064181e+11,2.194232e+11,2.543609e+11,...,1.701034e+11,2.483130e+11,1.853676e+11,1.890491e+11,1.786474e+11,5.939028e+11,7.031798e+11,3.648304e+11,7.343971e+11,4.498072e+11


In [157]:
train_processed.sum().sum()

4.62967405368569e+16